# Importing the libraries and files needed for the analysis

In [ ]:
 # Import the libraries

 import numpy as np

 import pandas as pd

In [ ]:
# Read the annual causes of death file and preview the data

death_cause = pd.read_csv('/content/annual-number-of-deaths-by-cause.csv')

death_cause.sample(5)

,Entity,Code,Year,Number of executions (Amnesty International),Deaths - Road injuries - Sex: Both - Age: All Ages (Number),Deaths - Cirrhosis and other chronic liver diseases - Sex: Both - Age: All Ages (Number),Deaths - Digestive diseases - Sex: Both - Age: All Ages (Number),Deaths - Tuberculosis - Sex: Both - Age: All Ages (Number),Deaths - HIV/AIDS - Sex: Both - Age: All Ages (Number),Deaths - Diarrheal diseases - Sex: Both - Age: All Ages (Number),Deaths - Intestinal infectious diseases - Sex: Both - Age: All Ages (Number),Deaths - Lower respiratory infections - Sex: Both - Age: All Ages (Number),Deaths - Meningitis - Sex: Both - Age: All Ages (Number),Deaths - Drowning - Sex: Both - Age: All Ages (Number),Deaths - Alzheimer disease and other dementias - Sex: Both - Age: All Ages (Number),Deaths - Parkinson disease - Sex: Both - Age: All Ages (Number),Deaths - Alcohol use disorders - Sex: Both - Age: All Ages (Number),Deaths - Drug use disorders - Sex: Both - Age: All Ages (Number),Deaths - Malaria - Sex: Both - Age: All Ages (Number),Deaths - Maternal disorders - Sex: Both - Age: All Ages (Number),Deaths - Neonatal disorders - Sex: Both - Age: All Ages (Number),Deaths - Nutritional deficiencies - Sex: Both - Age: All Ages (Number),Deaths - Diabetes mellitus - Sex: Both - Age: All Ages (Number),Deaths - Chronic kidney disease - Sex: Both - Age: All Ages (Number),Deaths - Chronic respiratory diseases - Sex: Both - Age: All Ages (Number),Deaths - Conflict and terrorism - Sex: Both - Age: All Ages (Number),Deaths - Hepatitis - Sex: Both - Age: All Ages (Number),Deaths - Neoplasms - Sex: Both - Age: All Ages (Number),"Deaths - Fire, heat, and hot substances - Sex: Both - Age: All Ages (Number)",Deaths - Poisonings - Sex: Both - Age: All Ages (Number),Deaths - Exposure to forces of nature - Sex: Both - Age: All Ages (Number),Deaths - Environmental heat and cold exposure - Sex: Both - Age: All Ages (Number),Deaths - Protein-energy malnutrition - Sex: Both - Age: All Ages (Number),Deaths - Cardiovascular diseases - Sex: Both - Age: All Ages (Number),Deaths - Self-harm - Sex: Both - Age: All Ages (Number),Deaths - Interpersonal violence - Sex: Both - Age: All Ages (Number),Terrorism (deaths)
5781,Switzerland,CHE,2015,NaN,345.335133,931.951919,2560.633238,32.203371,31.951279,221.378293,0.345859,1647.615976,28.531129,56.033962,7301.030877,850.023402,260.393296,197.890981,0.000000,7.189186,173.979370,74.246894,1099.672425,1647.493298,2856.898655,0.000000,77.913037,17981.277530,42.005389,12.211130,18.999999,17.115985,48.247000,2.104684e+04,1175.307453,54.951949,NaN
3651,Mali,MLI,1991,NaN,2707.641644,1824.028386,3417.284206,5711.199404,728.020690,23383.526470,632.003791,12109.373440,5937.245953,1167.152534,686.811461,126.157357,59.294079,26.155706,17708.780250,2563.103081,22256.242110,10486.039190,916.866285,1623.316073,3269.814059,230.999994,1074.972866,6083.826299,584.532871,196.642294,0.000000,123.955479,10205.399210,1.194311e+04,388.973883,576.964389,43.0
3446,Low SDI,NaN,2014,NaN,208261.864000,159916.654000,331042.431900,484039.082300,364903.151900,721506.040700,20877.361410,738693.606000,132557.083200,74354.565740,104884.458000,20032.493630,12439.524020,9272.217137,371247.046000,97927.717730,818334.126800,139088.968700,159423.549800,120937.286800,696193.125300,33695.845780,40172.141760,627949.189400,29835.041840,10636.062360,1581.717778,12002.730100,126257.322900,1.488729e+06,98165.647200,57301.364740,NaN
4076,Namibia,NAM,1995,NaN,452.249230,118.568779,413.526392,899.642982,2797.162296,1099.305643,1.864485,1103.106970,125.453158,73.151630,203.894737,28.809300,34.089107,25.389777,6.435075,104.781271,978.823986,203.976819,485.522870,205.064610,723.424296,0.000000,12.014842,1007.607719,82.366700,28.687820,0.000000,19.932655,195.662242,2.951354e+03,223.611804,366.534147,NaN
2926,Israel,ISR,2017,NaN,631.676349,707.313207,1807.963021,26.474081,41.439863,274.510664,NaN,2275.921656,35.910923,53.002941,4657.719654,567.637218,58.655084,70.

In [ ]:
# Read the World Bank metadata file and preview the data
# This file will help us in our analysis by helping to group the world countries in the causes of death file by region and income group

metadata=pd.read_excel('/content/WITSCountryProfile-Country_Indicator_ProductMetada-en.xlsx')

metadata.sample(5)

,Country Name,Country ISO3,Country Code,Long Name,Income Group,Lending Category,Region,Currency Unit,Other Groups,WTO Member
207,"Serbia, FR(Serbia/Montenegro)",SER,891,NaN,Others,NaN,NaN,NaN,NaN,No
219,South Georgia and the South Sa,SGS,239,NaN,Others,NaN,NaN,NaN,NaN,No
206,Senegal,SEN,686,Republic of Senegal,Lower middle income,IDA,Sub-Saharan Africa,CFA franc,HIPC,Yes
132,Latin America & Caribbean,LCN,LCN,Latin America & Caribbean (all income levels),Others,NaN,NaN,NaN,NaN,NaN
204,Sao Tome and Principe,STP,678,Democratic Republic of São Tomé and Principe,Lower middle income,IDA,Sub-Saharan Africa,São Tomé and Principe dobra,HIPC,No


# Preview information on the causes of death file

In [ ]:
# Check the number of missing data

num_rows = death_cause.shape[0] # Putting the [0] shows you want only rows.

death_cause_missing = num_rows - death_cause.count() 
death_cause_missing

Entity                                                                                         0
Code                                                                                        1141
Year                                                                                           0
Number of executions (Amnesty International)                                                6419
Deaths - Road injuries - Sex: Both - Age: All Ages (Number)                                  218
Deaths - Cirrhosis and other chronic liver diseases - Sex: Both - Age: All Ages (Number)     218
Deaths - Digestive diseases - Sex: Both - Age: All Ages (Number)                             218
Deaths - Tuberculosis - Sex: Both - Age: All Ages (Number)                                   218
Deaths - HIV/AIDS - Sex: Both - Age: All Ages (Number)                                       218
Deaths - Diarrheal diseases - Sex: Both - Age: All Ages (Number)                             218
Deaths - Intestinal infectious

In [ ]:
# Get the shape of the dataset
print(death_cause.shape)

(6686, 37)


# Preview information on the WorldBank metadata file

In [ ]:
# Check the number of missing data

num_rowsWB = metadata.shape[0] # Putting the [0] shows you want only rows.

metadata_missing = num_rowsWB - metadata.count() 
metadata_missing

Country Name          0
Country ISO3          0
Country Code          0
Long Name            53
Income Group          0
Lending Category    128
Region               66
Currency Unit        66
Other Groups        210
WTO Member           12
dtype: int64

In [ ]:
# Get the shape of the dataset
print(metadata.shape)


(269, 10)


# World Bank Data Validity

In [ ]:
# Delete irrelevant columns
# We start by dropping the irrelevant columns in the World Bank metadata dataset as this is the one that needs the 
# most cleaning before merging the two datasets

metadata.drop(["Country Code", "Long Name", "Lending Category", "Currency Unit", "Other Groups", "WTO Member"], axis = 1, inplace = True)

# Strip the column headers of leading and lagging spaces and convert them to lowercase
metadata.columns = metadata.columns.str.strip().str.lower()

# Rename the "country iso3" column to code, so that it can be uniform with the same header in the causes of death file

metadata.rename(columns = {'country iso3':'code'}, inplace = True) 

# Preview the dataset
metadata.sample(3)

,country name,code,income group,region
145,Maldives,MDV,Upper middle income,South Asia
35,Brunei,BRN,High income: nonOECD,East Asia & Pacific
20,Belgium,BEL,High income: OECD,Europe & Central Asia


# Causes of Death Data Validity

In [ ]:
# This dataset doesnt have any irrelevant columns as yet (we will drop the code column later after combining the 
# two datasets)
# As such, for now we start by stripping the column headers of leading and lagging spaces, converting them to lowercase and stripping the headers
# of any unnecessary header details
death_cause.columns = death_cause.columns.str.strip().str.lower().str.replace("(","").str.replace(")","").str.replace("amnesty international","").str.replace(" - sex: both - age: all ages number","").str.replace("deaths - ","")

# Preview the dataset
death_cause.head()

,entity,code,year,number of executions,road injuries,cirrhosis and other chronic liver diseases,digestive diseases,tuberculosis,hiv/aids,diarrheal diseases,intestinal infectious diseases,lower respiratory infections,meningitis,drowning,alzheimer disease and other dementias,parkinson disease,alcohol use disorders,drug use disorders,malaria,maternal disorders,neonatal disorders,nutritional deficiencies,diabetes mellitus,chronic kidney disease,chronic respiratory diseases,conflict and terrorism,hepatitis,neoplasms,"fire, heat, and hot substances",poisonings,exposure to forces of nature,environmental heat and cold exposure,protein-energy malnutrition,cardiovascular diseases,self-harm,interpersonal violence,terrorism deaths
0,Afghanistan,AFG,1990,NaN,4575.180311,1453.044977,3698.889864,4332.690583,32.098129,8817.260208,295.382206,22836.91235,6469.977091,1508.416465,1959.215373,324.828945,75.595221,78.254192,463.612423,2283.021996,18075.80029,1667.386134,3361.736409,3155.356340,5954.958582,1489.999290,793.670257,10061.87374,311.776082,170.742286,0.000000,226.789403,1607.703797,46498.08502,740.448500,1279.945985,12.0
1,Afghanistan,AFG,1991,NaN,4762.068567,1468.860745,3743.187336,4288.640464,37.524158,8588.463794,303.866598,22325.63393,6347.158764,1473.225021,1987.812878,328.452300,76.109641,81.284586,487.191614,2337.414404,18349.12997,1616.756233,3362.470217,3124.082631,6023.386323,3370.000153,788.536280,10171.33351,311.500538,172.032657,1349.999434,164.830417,1558.132316,46967.36104,762.245179,1524.641593,68.0
2,Afghanistan,AFG,1992,NaN,5531.136933,1524.528468,3894.366133,4475.257542,50.670559,8924.932420,317.750121,23205.28075,6659.741428,1537.813332,2025.102326,334.707212,81.138310,91.010197,521.714216,2781.918017,19982.14956,1678.557625,3445.705567,3192.278428,6217.245157,4343.997795,823.915867,10592.36758,338.614263,190.404111,614.000083,92.522183,1617.721576,48355.55851,856.912927,1684.808857,49.0
3,Afghanistan,AFG,1993,NaN,6613.963510,1598.643747,4136.479100,4909.050325,58.433049,11976.501320,333.978331,28229.71540,8068.386263,1714.815875,2064.827903,341.810023,87.514744,103.481991,675.657696,3336.796608,27291.23529,2001.122436,3563.236367,3343.974014,6468.245470,4097.000207,892.642586,11126.64465,382.401317,217.622614,109.999998,104.167461,1931.808553,50072.56390,975.603772,1906.959925,NaN
4,Afghanistan,AFG,1994,NaN,7341.204565,1649.977411,4318.692618,5352.634058,65.531176,13296.448480,348.419085,32652.29763,9432.845127,1949.699499,2101.020630,348.223321,90.464149,111.296397,782.445339,3558.597715,29862.68087,2434.014268,3639.707847,3480.299710,6678.495850,8959.000461,950.379553,11477.93757,417.717980,236.240573,160.000003,112.072652,2351.629206,51416.81544,1028.742412,2229.924043,22.0


# Merging the two datasets into one

In [ ]:
# Merge the two datasets

df_cleaned=death_cause.merge(metadata, how='left', on='code')  # merges on column "code"

df_cleaned.head()

,entity,code,year,number of executions,road injuries,cirrhosis and other chronic liver diseases,digestive diseases,tuberculosis,hiv/aids,diarrheal diseases,intestinal infectious diseases,lower respiratory infections,meningitis,drowning,alzheimer disease and other dementias,parkinson disease,alcohol use disorders,drug use disorders,malaria,maternal disorders,neonatal disorders,nutritional deficiencies,diabetes mellitus,chronic kidney disease,chronic respiratory diseases,conflict and terrorism,hepatitis,neoplasms,"fire, heat, and hot substances",poisonings,exposure to forces of nature,environmental heat and cold exposure,protein-energy malnutrition,cardiovascular diseases,self-harm,interpersonal violence,terrorism deaths,country name,income group,region
0,Afghanistan,AFG,1990,NaN,4575.180311,1453.044977,3698.889864,4332.690583,32.098129,8817.260208,295.382206,22836.91235,6469.977091,1508.416465,1959.215373,324.828945,75.595221,78.254192,463.612423,2283.021996,18075.80029,1667.386134,3361.736409,3155.356340,5954.958582,1489.999290,793.670257,10061.87374,311.776082,170.742286,0.000000,226.789403,1607.703797,46498.08502,740.448500,1279.945985,12.0,Afghanistan,Low income,South Asia
1,Afghanistan,AFG,1991,NaN,4762.068567,1468.860745,3743.187336,4288.640464,37.524158,8588.463794,303.866598,22325.63393,6347.158764,1473.225021,1987.812878,328.452300,76.109641,81.284586,487.191614,2337.414404,18349.12997,1616.756233,3362.470217,3124.082631,6023.386323,3370.000153,788.536280,10171.33351,311.500538,172.032657,1349.999434,164.830417,1558.132316,46967.36104,762.245179,1524.641593,68.0,Afghanistan,Low income,South Asia
2,Afghanistan,AFG,1992,NaN,5531.136933,1524.528468,3894.366133,4475.257542,50.670559,8924.932420,317.750121,23205.28075,6659.741428,1537.813332,2025.102326,334.707212,81.138310,91.010197,521.714216,2781.918017,19982.14956,1678.557625,3445.705567,3192.278428,6217.245157,4343.997795,823.915867,10592.36758,338.614263,190.404111,614.000083,92.522183,1617.721576,48355.55851,856.912927,1684.808857,49.0,Afghanistan,Low income,South Asia
3,Afghanistan,AFG,1993,NaN,6613.963510,1598.643747,4136.479100,4909.050325,58.433049,11976.501320,333.978331,28229.71540,8068.386263,1714.815875,2064.827903,341.810023,87.514744,103.481991,675.657696,3336.796608,27291.23529,2001.122436,3563.236367,3343.974014,6468.245470,4097.000207,892.642586,11126.64465,382.401317,217.622614,109.999998,104.167461,1931.808553,50072.56390,975.603772,1906.959925,NaN,Afghanistan,Low income,South Asia
4,Afghanistan,AFG,1994,NaN,7341.204565,1649.977411,4318.692618,5352.634058,65.531176,13296.448480,348.419085,32652.29763,9432.845127,1949.699499,2101.020630,348.223321,90.464149,111.296397,782.445339,3558.597715,29862.68087,2434.014268,3639.707847,3480.299710,6678.495850,8959.000461,950.379553,11477.93757,417.717980,236.240573,160.000003,112.072652,2351.629206,51416.81544,1028.742412,2229.924043,22.0,Afghanistan,Low income,South Asia


In [ ]:
# Check the rows where region is missing to see whether they are regions we should drop or whether further investigation should be made

df_missing_data=df_cleaned[df_cleaned['region'].isnull()]

print(df_missing_data.shape)

# The missing dataset above gives us all the rows based on missing a value in the column region.
# The code below then sorts out the dataset and returns for us only the unique values based on the column "entity name". This means that for all instances
# of the entity name, we will get only one row with that entity name(i.e. country name), greatly minimizing our dataset and allowing us to interogate
# the dataset quickly
df_missing_data_2 = df_missing_data.drop_duplicates(subset = ["entity"])

# previews our new dataset. Showing that there are 68 rows
print(df_missing_data_2.shape)
df_missing_data_2


(1493, 40)
(68, 40)


,entity,code,year,number of executions,road injuries,cirrhosis and other chronic liver diseases,digestive diseases,tuberculosis,hiv/aids,diarrheal diseases,intestinal infectious diseases,lower respiratory infections,meningitis,drowning,alzheimer disease and other dementias,parkinson disease,alcohol use disorders,drug use disorders,malaria,maternal disorders,neonatal disorders,nutritional deficiencies,diabetes mellitus,chronic kidney disease,chronic respiratory diseases,conflict and terrorism,hepatitis,neoplasms,"fire, heat, and hot substances",poisonings,exposure to forces of nature,environmental heat and cold exposure,protein-energy malnutrition,cardiovascular diseases,self-harm,interpersonal violence,terrorism deaths,country name,income group,region
112,Andean Latin America,NaN,1990,NaN,8.812550e+03,6020.812984,1.401370e+04,1.312591e+04,752.607924,1.346002e+04,12.454473,3.629138e+04,1526.020927,3125.268433,5.101489e+03,771.341222,1083.194788,302.171250,116.137437,2388.579254,2.261069e+04,7101.711369,3453.312727,5281.070417,7.082953e+03,2376.999911,323.907830,2.574643e+04,1209.431120,678.463102,297.999995,324.285229,6575.288360,4.034033e+04,1690.350879,2996.162633,NaN,NaN,NaN,NaN
140,Andorra,AND,1990,NaN,5.245106e+00,6.323226,1.412365e+01,4.369409e-01,1.060716,1.971131e-01,0.002202,1.215719e+01,0.371722,0.519074,1.242265e+01,2.426882,0.748875,1.067791,0.000000,0.040999,1.350708e+00,0.078023,2.660915,2.154288,1.414388e+01,0.000000,0.092494,8.420647e+01,0.566660,0.206601,0.000000,0.190192,0.035425,9.822218e+01,7.093039,0.659100,NaN,Andorra,Others,NaN
280,Australasia,NaN,1990,NaN,3.472617e+03,1533.671516,4.644609e+03,1.301915e+02,430.842064,4.591326e+01,0.375149,2.551961e+03,143.913788,372.243400,7.624496e+03,1061.228087,292.028709,481.889875,0.000000,26.441674,1.106328e+03,63.233410,2449.511429,1952.836532,8.546839e+03,2.981484,61.233672,3.788214e+04,213.383833,57.804513,18.797859,51.558433,34.135031,6.251604e+04,2830.250713,433.545837,NaN,NaN,NaN,NaN
308,Australasia & Oceania,NaN,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN
755,Bosnia-Herzegovina,NaN,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6518,Western Sub-Saharan Africa,NaN,1990,NaN,3.782754e+04,53268.437720,8.588299e+04,1.444418e+05,43729.987310,3.863790e+05,13724.571320,3.686074e+05,99754.034740,15584.575820,1.923855e+04,3223.658644,1899.235461,958.018291,273704.644500,34156.659460,2.753319e+05,89371.777670,16080.095920,27506.555140,4.596592e+04,5123.146157,9264.628197,1.076908e+05,8234.607615,3047.658641,0.000000,1799.722984,86294.558140,2.116960e+05,10498.056940,9865.903842,NaN,NaN,NaN,NaN
6546,World,OWID_WRL,1990,NaN,1.150717e+06,898990.854800,1.714979e+06,1.607709e+06,348599.565300,2.578292e+06,215726.866700,3.415941e+06,462419.725400,531956.029100,1.022007e+06,147985.422100,112572.745400,70847.121630,672540.219200,325394.686000,2.986076e+06,568075.207000,607428.037500,613746.377800,3.317205e+06,100684.186600,146879.949500,5.753106e+06,146482.694700,77542.741650,50241.334720,58519.169680,511854.722900,1.194153e+07,758518.989100,353134.788500,7148.0,NaN,NaN,NaN
6574,World (excluding China),NaN,2007,1248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6612,Yugoslavia,OWID_YGS,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [ ]:
# Take the 68 rows to excel to better analyze the missing dataset
df_missing_data_2.to_excel('missing_data.xls')

# Reload the cleaned metadata dataset and merge

In [ ]:
# The step above took the missing data to excel. Here, we analyzed the data that did not merge properly due to non-matching codes in the two sets.
# We also analyzed the data that legitimately needed to be dropped due to one reason or another. This table is presented in our detailed report.
# For the data that didn't merge due to the fact that the coding in the metadata file was different from the coding in the causes of death file,
# we ensured that we changed the coding in the metadata file to match the coding in the causes of death file. The cleaned up metadata file is 
# now imported so as to merge the two datasets.

metadata_cleaned=pd.read_excel("/content/World Bank Metadata Cleaned Regions File.xlsx")

metadata_cleaned.head()

,Country Name,Country ISO3,Country Code,Long Name,Income Group,Lending Category,Region,Currency Unit,Other Groups,WTO Member
0,Afghanistan,AFG,004,Islamic State of Afghanistan,Low income,IDA,South Asia,Afghan afghani,HIPC,No
1,Albania,ALB,008,Republic of Albania,Upper middle income,IBRD,Europe & Central Asia,Albanian lek,NaN,Yes
2,Algeria,DZA,012,People's Democratic Republic of Algeria,Upper middle income,IBRD,Middle East & North Africa,Algerian dinar,NaN,No
3,American Samoa,ASM,016,American Samoa,Upper middle income,NaN,East Asia & Pacific,U.S. dollar,NaN,No
4,Andorra,AND,020,NaN,Others,NaN,Europe & Central Asia,NaN,NaN,No


In [ ]:
# Repeat the clean-up of the metadata file 
# Delete irrelevant columns
# We start by dropping the irrelevant columns in the World Bank metadata dataset as this is the one that needs the 
# most cleaning before merging the two datasets

metadata_cleaned.drop(["Country Code", "Long Name", "Lending Category", "Currency Unit", "Other Groups", "WTO Member"], axis = 1, inplace = True)

# Strip the column headers of leading and lagging spaces and convert them to lowercase
metadata_cleaned.columns = metadata_cleaned.columns.str.strip().str.lower()

# Rename the "country iso3" column to code, so that it can be uniform with the same header in the causes of death file

metadata_cleaned.rename(columns = {'country iso3':'code'}, inplace = True) 

# Preview the dataset
metadata_cleaned.sample(3)

,country name,code,income group,region
47,Chile,CHL,High income: OECD,Latin America & Caribbean
161,Montserrat,MSR,Others,NaN
199,Rwanda,RWA,Low income,Sub-Saharan Africa


In [ ]:
# Merge the two datasets
# Here, we take the causes of death as cleaned in this collaboratory notebook and merge it with the cleaned Metadata notebook that has been uploaded
# after making the necessary changes to the country codes in Excel

df_cleaned=death_cause.merge(metadata_cleaned, how='left', on='code')  # merges on column "code"

df_cleaned.head()

,entity,code,year,number of executions,road injuries,cirrhosis and other chronic liver diseases,digestive diseases,tuberculosis,hiv/aids,diarrheal diseases,intestinal infectious diseases,lower respiratory infections,meningitis,drowning,alzheimer disease and other dementias,parkinson disease,alcohol use disorders,drug use disorders,malaria,maternal disorders,neonatal disorders,nutritional deficiencies,diabetes mellitus,chronic kidney disease,chronic respiratory diseases,conflict and terrorism,hepatitis,neoplasms,"fire, heat, and hot substances",poisonings,exposure to forces of nature,environmental heat and cold exposure,protein-energy malnutrition,cardiovascular diseases,self-harm,interpersonal violence,terrorism deaths,country name,income group,region
0,Afghanistan,AFG,1990,NaN,4575.180311,1453.044977,3698.889864,4332.690583,32.098129,8817.260208,295.382206,22836.91235,6469.977091,1508.416465,1959.215373,324.828945,75.595221,78.254192,463.612423,2283.021996,18075.80029,1667.386134,3361.736409,3155.356340,5954.958582,1489.999290,793.670257,10061.87374,311.776082,170.742286,0.000000,226.789403,1607.703797,46498.08502,740.448500,1279.945985,12.0,Afghanistan,Low income,South Asia
1,Afghanistan,AFG,1991,NaN,4762.068567,1468.860745,3743.187336,4288.640464,37.524158,8588.463794,303.866598,22325.63393,6347.158764,1473.225021,1987.812878,328.452300,76.109641,81.284586,487.191614,2337.414404,18349.12997,1616.756233,3362.470217,3124.082631,6023.386323,3370.000153,788.536280,10171.33351,311.500538,172.032657,1349.999434,164.830417,1558.132316,46967.36104,762.245179,1524.641593,68.0,Afghanistan,Low income,South Asia
2,Afghanistan,AFG,1992,NaN,5531.136933,1524.528468,3894.366133,4475.257542,50.670559,8924.932420,317.750121,23205.28075,6659.741428,1537.813332,2025.102326,334.707212,81.138310,91.010197,521.714216,2781.918017,19982.14956,1678.557625,3445.705567,3192.278428,6217.245157,4343.997795,823.915867,10592.36758,338.614263,190.404111,614.000083,92.522183,1617.721576,48355.55851,856.912927,1684.808857,49.0,Afghanistan,Low income,South Asia
3,Afghanistan,AFG,1993,NaN,6613.963510,1598.643747,4136.479100,4909.050325,58.433049,11976.501320,333.978331,28229.71540,8068.386263,1714.815875,2064.827903,341.810023,87.514744,103.481991,675.657696,3336.796608,27291.23529,2001.122436,3563.236367,3343.974014,6468.245470,4097.000207,892.642586,11126.64465,382.401317,217.622614,109.999998,104.167461,1931.808553,50072.56390,975.603772,1906.959925,NaN,Afghanistan,Low income,South Asia
4,Afghanistan,AFG,1994,NaN,7341.204565,1649.977411,4318.692618,5352.634058,65.531176,13296.448480,348.419085,32652.29763,9432.845127,1949.699499,2101.020630,348.223321,90.464149,111.296397,782.445339,3558.597715,29862.68087,2434.014268,3639.707847,3480.299710,6678.495850,8959.000461,950.379553,11477.93757,417.717980,236.240573,160.000003,112.072652,2351.629206,51416.81544,1028.742412,2229.924043,22.0,Afghanistan,Low income,South Asia


In [ ]:
# Check the rows where region column is blank so as to see whether the analysis done above has made any changes to our dataset

df_missing_data=df_cleaned[df_cleaned['region'].isnull()]

print(df_missing_data.shape)

# The missing dataset above gives us all the rows based on missing a value in the column region. The output shows that we have 1,350 rows with a null value
# in the column "region". This has reduced from the earlier figure of 1,493 rows.
# The code below then sorts out the dataset and returns for us only the unique values based on the column "entity name". This means that for all instances
# of the entity name, we will get only one row with that entity name(i.e. country name), greatly minimizing our dataset and allowing us to interogate
# the dataset quickly
df_missing_data_2 = df_missing_data.drop_duplicates(subset = ["entity"])

# previews our new dataset showing us that there are now 61 unique rows with missing data under the column "Region".
print(df_missing_data_2.shape)
df_missing_data_2

(1350, 40)
(61, 40)


,entity,code,year,number of executions,road injuries,cirrhosis and other chronic liver diseases,digestive diseases,tuberculosis,hiv/aids,diarrheal diseases,intestinal infectious diseases,lower respiratory infections,meningitis,drowning,alzheimer disease and other dementias,parkinson disease,alcohol use disorders,drug use disorders,malaria,maternal disorders,neonatal disorders,nutritional deficiencies,diabetes mellitus,chronic kidney disease,chronic respiratory diseases,conflict and terrorism,hepatitis,neoplasms,"fire, heat, and hot substances",poisonings,exposure to forces of nature,environmental heat and cold exposure,protein-energy malnutrition,cardiovascular diseases,self-harm,interpersonal violence,terrorism deaths,country name,income group,region
112,Andean Latin America,NaN,1990,NaN,8.812550e+03,6020.812984,1.401370e+04,1.312591e+04,752.607924,1.346002e+04,12.454473,3.629138e+04,1526.020927,3125.268433,5.101489e+03,771.341222,1083.194788,302.171250,116.137437,2388.579254,2.261069e+04,7101.711369,3453.312727,5281.070417,7.082953e+03,2376.999911,323.907830,2.574643e+04,1209.431120,678.463102,297.999995,324.285229,6575.288360,4.034033e+04,1690.350879,2996.162633,NaN,NaN,NaN,NaN
280,Australasia,NaN,1990,NaN,3.472617e+03,1533.671516,4.644609e+03,1.301915e+02,430.842064,4.591326e+01,0.375149,2.551961e+03,143.913788,372.243400,7.624496e+03,1061.228087,292.028709,481.889875,0.000000,26.441674,1.106328e+03,63.233410,2449.511429,1952.836532,8.546839e+03,2.981484,61.233672,3.788214e+04,213.383833,57.804513,18.797859,51.558433,34.135031,6.251604e+04,2830.250713,433.545837,NaN,NaN,NaN,NaN
308,Australasia & Oceania,NaN,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN
755,Bosnia-Herzegovina,NaN,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,NaN,NaN,NaN
1056,Caribbean,NaN,1990,NaN,9.794945e+03,5453.339219,1.061148e+04,3.801083e+03,4216.162055,1.501010e+04,11.507162,1.649923e+04,3241.491068,2760.573088,7.114363e+03,1026.358603,756.165214,132.814614,45.859122,1594.725656,1.366653e+04,3948.850611,10165.420070,4942.508716,7.358353e+03,59.019614,280.196790,3.451674e+04,1283.363819,485.715656,4.141727,233.914632,3687.667676,8.239703e+04,4050.504154,4759.566543,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6518,Western Sub-Saharan Africa,NaN,1990,NaN,3.782754e+04,53268.437720,8.588299e+04,1.444418e+05,43729.987310,3.863790e+05,13724.571320,3.686074e+05,99754.034740,15584.575820,1.923855e+04,3223.658644,1899.235461,958.018291,273704.644500,34156.659460,2.753319e+05,89371.777670,16080.095920,27506.555140,4.596592e+04,5123.146157,9264.628197,1.076908e+05,8234.607615,3047.658641,0.000000,1799.722984,86294.558140,2.116960e+05,10498.056940,9865.903842,NaN,NaN,NaN,NaN
6546,World,OWID_WRL,1990,NaN,1.150717e+06,898990.854800,1.714979e+06,1.607709e+06,348599.565300,2.578292e+06,215726.866700,3.415941e+06,462419.725400,531956.029100,1.022007e+06,147985.422100,112572.745400,70847.121630,672540.219200,325394.686000,2.986076e+06,568075.207000,607428.037500,613746.377800,3.317205e+06,100684.186600,146879.949500,5.753106e+06,146482.694700,77542.741650,50241.334720,58519.169680,511854.722900,1.194153e+07,758518.989100,353134.788500,7148.0,NaN,NaN,NaN
6574,World (excluding China),NaN,2007,1248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6612,Yugoslavia,OWID_YGS,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


# Continue the analysis by selecting rows with non-null values under the column "Region" as these are the rows of interest in our analysis

In [ ]:
# Select rows with non-null values from region column of the the df_cleaned dataset above and assign these non-null value rows 
# to a new dataset called df_cleaned_data. This new dataset contains the vaues that are important to us going forward.

df_cleaned_data=df_cleaned[df_cleaned['region'].notnull()]

# Check the shape of the dataset
print(df_cleaned_data.shape)

#Preview the dataset
df_cleaned_data.head()

(5336, 40)


,entity,code,year,number of executions,road injuries,cirrhosis and other chronic liver diseases,digestive diseases,tuberculosis,hiv/aids,diarrheal diseases,intestinal infectious diseases,lower respiratory infections,meningitis,drowning,alzheimer disease and other dementias,parkinson disease,alcohol use disorders,drug use disorders,malaria,maternal disorders,neonatal disorders,nutritional deficiencies,diabetes mellitus,chronic kidney disease,chronic respiratory diseases,conflict and terrorism,hepatitis,neoplasms,"fire, heat, and hot substances",poisonings,exposure to forces of nature,environmental heat and cold exposure,protein-energy malnutrition,cardiovascular diseases,self-harm,interpersonal violence,terrorism deaths,country name,income group,region
0,Afghanistan,AFG,1990,NaN,4575.180311,1453.044977,3698.889864,4332.690583,32.098129,8817.260208,295.382206,22836.91235,6469.977091,1508.416465,1959.215373,324.828945,75.595221,78.254192,463.612423,2283.021996,18075.80029,1667.386134,3361.736409,3155.356340,5954.958582,1489.999290,793.670257,10061.87374,311.776082,170.742286,0.000000,226.789403,1607.703797,46498.08502,740.448500,1279.945985,12.0,Afghanistan,Low income,South Asia
1,Afghanistan,AFG,1991,NaN,4762.068567,1468.860745,3743.187336,4288.640464,37.524158,8588.463794,303.866598,22325.63393,6347.158764,1473.225021,1987.812878,328.452300,76.109641,81.284586,487.191614,2337.414404,18349.12997,1616.756233,3362.470217,3124.082631,6023.386323,3370.000153,788.536280,10171.33351,311.500538,172.032657,1349.999434,164.830417,1558.132316,46967.36104,762.245179,1524.641593,68.0,Afghanistan,Low income,South Asia
2,Afghanistan,AFG,1992,NaN,5531.136933,1524.528468,3894.366133,4475.257542,50.670559,8924.932420,317.750121,23205.28075,6659.741428,1537.813332,2025.102326,334.707212,81.138310,91.010197,521.714216,2781.918017,19982.14956,1678.557625,3445.705567,3192.278428,6217.245157,4343.997795,823.915867,10592.36758,338.614263,190.404111,614.000083,92.522183,1617.721576,48355.55851,856.912927,1684.808857,49.0,Afghanistan,Low income,South Asia
3,Afghanistan,AFG,1993,NaN,6613.963510,1598.643747,4136.479100,4909.050325,58.433049,11976.501320,333.978331,28229.71540,8068.386263,1714.815875,2064.827903,341.810023,87.514744,103.481991,675.657696,3336.796608,27291.23529,2001.122436,3563.236367,3343.974014,6468.245470,4097.000207,892.642586,11126.64465,382.401317,217.622614,109.999998,104.167461,1931.808553,50072.56390,975.603772,1906.959925,NaN,Afghanistan,Low income,South Asia
4,Afghanistan,AFG,1994,NaN,7341.204565,1649.977411,4318.692618,5352.634058,65.531176,13296.448480,348.419085,32652.29763,9432.845127,1949.699499,2101.020630,348.223321,90.464149,111.296397,782.445339,3558.597715,29862.68087,2434.014268,3639.707847,3480.299710,6678.495850,8959.000461,950.379553,11477.93757,417.717980,236.240573,160.000003,112.072652,2351.629206,51416.81544,1028.742412,2229.924043,22.0,Afghanistan,Low income,South Asia


In [ ]:
# Continue cleaning the data by selecting rows with non-null values in the income group
# Here, we create a new dataframe and it is to sort out data from the df_cleaned_data dataframe. This two step process ensures that we clean our data
# of both non-null values from the region and income group columns.
df_cleaned_data2=df_cleaned_data[df_cleaned_data['income group'].notnull()]

# Check the shape of the dataset. NB: The shape of teh dataset is the same as the dataset from df_cleaned_data, showing that the df_cleaned_data dataset
# did not contain any null values in the income group column.
print(df_cleaned_data2.shape)

#Preview the dataset
df_cleaned_data2.head()


(5336, 40)


,entity,code,year,number of executions,road injuries,cirrhosis and other chronic liver diseases,digestive diseases,tuberculosis,hiv/aids,diarrheal diseases,intestinal infectious diseases,lower respiratory infections,meningitis,drowning,alzheimer disease and other dementias,parkinson disease,alcohol use disorders,drug use disorders,malaria,maternal disorders,neonatal disorders,nutritional deficiencies,diabetes mellitus,chronic kidney disease,chronic respiratory diseases,conflict and terrorism,hepatitis,neoplasms,"fire, heat, and hot substances",poisonings,exposure to forces of nature,environmental heat and cold exposure,protein-energy malnutrition,cardiovascular diseases,self-harm,interpersonal violence,terrorism deaths,country name,income group,region
0,Afghanistan,AFG,1990,NaN,4575.180311,1453.044977,3698.889864,4332.690583,32.098129,8817.260208,295.382206,22836.91235,6469.977091,1508.416465,1959.215373,324.828945,75.595221,78.254192,463.612423,2283.021996,18075.80029,1667.386134,3361.736409,3155.356340,5954.958582,1489.999290,793.670257,10061.87374,311.776082,170.742286,0.000000,226.789403,1607.703797,46498.08502,740.448500,1279.945985,12.0,Afghanistan,Low income,South Asia
1,Afghanistan,AFG,1991,NaN,4762.068567,1468.860745,3743.187336,4288.640464,37.524158,8588.463794,303.866598,22325.63393,6347.158764,1473.225021,1987.812878,328.452300,76.109641,81.284586,487.191614,2337.414404,18349.12997,1616.756233,3362.470217,3124.082631,6023.386323,3370.000153,788.536280,10171.33351,311.500538,172.032657,1349.999434,164.830417,1558.132316,46967.36104,762.245179,1524.641593,68.0,Afghanistan,Low income,South Asia
2,Afghanistan,AFG,1992,NaN,5531.136933,1524.528468,3894.366133,4475.257542,50.670559,8924.932420,317.750121,23205.28075,6659.741428,1537.813332,2025.102326,334.707212,81.138310,91.010197,521.714216,2781.918017,19982.14956,1678.557625,3445.705567,3192.278428,6217.245157,4343.997795,823.915867,10592.36758,338.614263,190.404111,614.000083,92.522183,1617.721576,48355.55851,856.912927,1684.808857,49.0,Afghanistan,Low income,South Asia
3,Afghanistan,AFG,1993,NaN,6613.963510,1598.643747,4136.479100,4909.050325,58.433049,11976.501320,333.978331,28229.71540,8068.386263,1714.815875,2064.827903,341.810023,87.514744,103.481991,675.657696,3336.796608,27291.23529,2001.122436,3563.236367,3343.974014,6468.245470,4097.000207,892.642586,11126.64465,382.401317,217.622614,109.999998,104.167461,1931.808553,50072.56390,975.603772,1906.959925,NaN,Afghanistan,Low income,South Asia
4,Afghanistan,AFG,1994,NaN,7341.204565,1649.977411,4318.692618,5352.634058,65.531176,13296.448480,348.419085,32652.29763,9432.845127,1949.699499,2101.020630,348.223321,90.464149,111.296397,782.445339,3558.597715,29862.68087,2434.014268,3639.707847,3480.299710,6678.495850,8959.000461,950.379553,11477.93757,417.717980,236.240573,160.000003,112.072652,2351.629206,51416.81544,1028.742412,2229.924043,22.0,Afghanistan,Low income,South Asia


In [ ]:
# Drop the irrelevant column country name since it has the same values as entity. 

df_cleaned_data2.drop(["country name"], axis = 1, inplace = True)

# Thereafter, change the entity column name to country name

df_cleaned_data2.rename(columns = {'entity':'country'}, inplace = True)

df_cleaned_data2.head()

,country,code,year,number of executions,road injuries,cirrhosis and other chronic liver diseases,digestive diseases,tuberculosis,hiv/aids,diarrheal diseases,intestinal infectious diseases,lower respiratory infections,meningitis,drowning,alzheimer disease and other dementias,parkinson disease,alcohol use disorders,drug use disorders,malaria,maternal disorders,neonatal disorders,nutritional deficiencies,diabetes mellitus,chronic kidney disease,chronic respiratory diseases,conflict and terrorism,hepatitis,neoplasms,"fire, heat, and hot substances",poisonings,exposure to forces of nature,environmental heat and cold exposure,protein-energy malnutrition,cardiovascular diseases,self-harm,interpersonal violence,terrorism deaths,income group,region
0,Afghanistan,AFG,1990,NaN,4575.180311,1453.044977,3698.889864,4332.690583,32.098129,8817.260208,295.382206,22836.91235,6469.977091,1508.416465,1959.215373,324.828945,75.595221,78.254192,463.612423,2283.021996,18075.80029,1667.386134,3361.736409,3155.356340,5954.958582,1489.999290,793.670257,10061.87374,311.776082,170.742286,0.000000,226.789403,1607.703797,46498.08502,740.448500,1279.945985,12.0,Low income,South Asia
1,Afghanistan,AFG,1991,NaN,4762.068567,1468.860745,3743.187336,4288.640464,37.524158,8588.463794,303.866598,22325.63393,6347.158764,1473.225021,1987.812878,328.452300,76.109641,81.284586,487.191614,2337.414404,18349.12997,1616.756233,3362.470217,3124.082631,6023.386323,3370.000153,788.536280,10171.33351,311.500538,172.032657,1349.999434,164.830417,1558.132316,46967.36104,762.245179,1524.641593,68.0,Low income,South Asia
2,Afghanistan,AFG,1992,NaN,5531.136933,1524.528468,3894.366133,4475.257542,50.670559,8924.932420,317.750121,23205.28075,6659.741428,1537.813332,2025.102326,334.707212,81.138310,91.010197,521.714216,2781.918017,19982.14956,1678.557625,3445.705567,3192.278428,6217.245157,4343.997795,823.915867,10592.36758,338.614263,190.404111,614.000083,92.522183,1617.721576,48355.55851,856.912927,1684.808857,49.0,Low income,South Asia
3,Afghanistan,AFG,1993,NaN,6613.963510,1598.643747,4136.479100,4909.050325,58.433049,11976.501320,333.978331,28229.71540,8068.386263,1714.815875,2064.827903,341.810023,87.514744,103.481991,675.657696,3336.796608,27291.23529,2001.122436,3563.236367,3343.974014,6468.245470,4097.000207,892.642586,11126.64465,382.401317,217.622614,109.999998,104.167461,1931.808553,50072.56390,975.603772,1906.959925,NaN,Low income,South Asia
4,Afghanistan,AFG,1994,NaN,7341.204565,1649.977411,4318.692618,5352.634058,65.531176,13296.448480,348.419085,32652.29763,9432.845127,1949.699499,2101.020630,348.223321,90.464149,111.296397,782.445339,3558.597715,29862.68087,2434.014268,3639.707847,3480.299710,6678.495850,8959.000461,950.379553,11477.93757,417.717980,236.240573,160.000003,112.072652,2351.629206,51416.81544,1028.742412,2229.924043,22.0,Low income,South Asia


In [ ]:
# Check the number of missing data

num_rows = df_cleaned_data2.shape[0] # Putting the [0] shows you want only rows.

df_cleaned_data2_missing = num_rows - df_cleaned_data2.count() 
df_cleaned_data2_missing

country                                          0
code                                             0
year                                             0
number of executions                          5096
road injuries                                   16
cirrhosis and other chronic liver diseases      16
digestive diseases                              16
tuberculosis                                    16
hiv/aids                                        16
diarrheal diseases                              16
intestinal infectious diseases                 206
lower respiratory infections                    16
meningitis                                      16
drowning                                        16
alzheimer disease and other dementias           16
parkinson disease                               16
alcohol use disorders                           16
drug use disorders                              16
malaria                                         16
maternal disorders             

In [ ]:
# Since the columns country, code, year, income group, and region all have non-null values, we can go ahead and fill in the entire missing dataset with Zeros.
# NB: If the columns above had non-null values, we would have had to first fill them up with relevant data so as not to fill them up with zeros.
# Alternatively, we could have filled in the columns with zeros one by one instead of a blanket fill of the whole dataset as done below.

df_cleaned_data2.fillna(0, inplace=True)

# Checking if there is any missing value across each column
df_cleaned_data2.isnull().any()

country                                       False
code                                          False
year                                          False
number of executions                          False
road injuries                                 False
cirrhosis and other chronic liver diseases    False
digestive diseases                            False
tuberculosis                                  False
hiv/aids                                      False
diarrheal diseases                            False
intestinal infectious diseases                False
lower respiratory infections                  False
meningitis                                    False
drowning                                      False
alzheimer disease and other dementias         False
parkinson disease                             False
alcohol use disorders                         False
drug use disorders                            False
malaria                                       False
maternal dis

In [ ]:
# Check the datatypes for each column
df_cleaned_data2.dtypes

country                                        object
code                                           object
year                                            int64
number of executions                           object
road injuries                                 float64
cirrhosis and other chronic liver diseases    float64
digestive diseases                            float64
tuberculosis                                  float64
hiv/aids                                      float64
diarrheal diseases                            float64
intestinal infectious diseases                float64
lower respiratory infections                  float64
meningitis                                    float64
drowning                                      float64
alzheimer disease and other dementias         float64
parkinson disease                             float64
alcohol use disorders                         float64
drug use disorders                            float64
malaria                     

In [ ]:
# Convert number of executions column to float

df_cleaned_data2['number of executions '] = pd.to_numeric(df_cleaned_data2['number of executions '], errors='coerce')

df_cleaned_data2.dtypes


country                                        object
code                                           object
year                                            int64
number of executions                          float64
road injuries                                 float64
cirrhosis and other chronic liver diseases    float64
digestive diseases                            float64
tuberculosis                                  float64
hiv/aids                                      float64
diarrheal diseases                            float64
intestinal infectious diseases                float64
lower respiratory infections                  float64
meningitis                                    float64
drowning                                      float64
alzheimer disease and other dementias         float64
parkinson disease                             float64
alcohol use disorders                         float64
drug use disorders                            float64
malaria                     

In [ ]:
# Export to Excel
df_cleaned_data2.to_excel('causes of death cleaned.xls')

# Data Analysis

# Question 1: What are the overall leading causes of death in the World?



In [ ]:
#remove years below 2000
df_cleaned_data2 = df_cleaned_data2[df_cleaned_data2['year'] > 2000] 
df_cleaned_data2

In [ ]:
# Determine the cummulative top 10 leading causes of death in the world

df_cleaned_data2[addition_columns].sum(axis=0).nlargest(10)

cardiovascular diseases                  2.607271e+08
neoplasms                                1.355094e+08
chronic respiratory diseases             5.973906e+07
lower respiratory infections             4.412236e+07
neonatal disorders                       3.703512e+07
digestive diseases                       3.550052e+07
alzheimer disease and other dementias    3.202197e+07
diarrheal diseases                       3.014191e+07
hiv/aids                                 2.613993e+07
tuberculosis                             2.210918e+07
dtype: float64

# Question 2: Which year experienced the most deaths?





In [ ]:
# To answer this question, we first need to write code that will add all the relevant columns per year and return to us the total deaths
# per year per country. To do this, we forst have to specify the list of columns to be included in the sum. We do this in the code below

addition_columns = list(df_cleaned_data2[['number of executions ', 'road injuries', 'cirrhosis and other chronic liver diseases', 'digestive diseases', 'tuberculosis',
                                     'hiv/aids', 'diarrheal diseases', 'intestinal infectious diseases', 'lower respiratory infections', 'meningitis', 
                                     'drowning', 'alzheimer disease and other dementias', 'parkinson disease', 'alcohol use disorders', 'drug use disorders', 
                                     'malaria', 'maternal disorders', 'neonatal disorders', 'nutritional deficiencies', 'diabetes mellitus', 'chronic kidney disease', 
                                     'chronic respiratory diseases', 'conflict and terrorism', 'hepatitis', 'neoplasms', 'fire, heat, and hot substances', 
                                     'poisonings', 'exposure to forces of nature', 'environmental heat and cold exposure', 'protein-energy malnutrition', 
                                     'cardiovascular diseases', 'self-harm', 'interpersonal violence', 'terrorism deaths']])

# Check that the columns have been called out correctly
addition_columns

['number of executions ',
 'road injuries',
 'cirrhosis and other chronic liver diseases',
 'digestive diseases',
 'tuberculosis',
 'hiv/aids',
 'diarrheal diseases',
 'intestinal infectious diseases',
 'lower respiratory infections',
 'meningitis',
 'drowning',
 'alzheimer disease and other dementias',
 'parkinson disease',
 'alcohol use disorders',
 'drug use disorders',
 'malaria',
 'maternal disorders',
 'neonatal disorders',
 'nutritional deficiencies',
 'diabetes mellitus',
 'chronic kidney disease',
 'chronic respiratory diseases',
 'conflict and terrorism',
 'hepatitis',
 'neoplasms',
 'fire, heat, and hot substances',
 'poisonings',
 'exposure to forces of nature',
 'environmental heat and cold exposure',
 'protein-energy malnutrition',
 'cardiovascular diseases',
 'self-harm',
 'interpersonal violence',
 'terrorism deaths']

In [ ]:
df_cleaned_data2["total deaths"] = df_cleaned_data2[addition_columns].sum(axis=1)

df_cleaned_data2.head(3)

,country,code,year,number of executions,road injuries,cirrhosis and other chronic liver diseases,digestive diseases,tuberculosis,hiv/aids,diarrheal diseases,intestinal infectious diseases,lower respiratory infections,meningitis,drowning,alzheimer disease and other dementias,parkinson disease,alcohol use disorders,drug use disorders,malaria,maternal disorders,neonatal disorders,nutritional deficiencies,diabetes mellitus,chronic kidney disease,chronic respiratory diseases,conflict and terrorism,hepatitis,neoplasms,"fire, heat, and hot substances",poisonings,exposure to forces of nature,environmental heat and cold exposure,protein-energy malnutrition,cardiovascular diseases,self-harm,interpersonal violence,terrorism deaths,income group,region,total deaths
11,Afghanistan,AFG,2001,0.0,7588.020991,1753.820705,4613.383954,5223.224733,94.691588,12136.21073,359.769786,30505.52451,9477.582051,2160.373879,2267.435697,378.218095,90.293732,159.495285,952.404384,3975.929836,29730.89428,2052.330272,4147.689517,3468.138292,7058.555969,5727.006858,1074.898095,12830.90154,449.031594,254.529894,4.000000,463.657061,1966.565922,54850.34660,1191.548668,2503.286991,174.0,Low income,South Asia,209683.761509
12,Afghanistan,AFG,2002,0.0,7641.680609,1761.552916,4622.104238,5120.447533,110.857059,11363.24605,364.035375,28909.06415,8335.205303,2111.261248,2295.084048,383.096750,90.896730,169.192335,951.399078,4271.732412,28324.18976,1943.242372,4257.009130,3470.397898,7037.320903,985.000023,1059.773351,13069.17457,447.978499,257.019972,1285.999798,140.765586,1857.651707,54810.28059,1238.790835,2903.788508,74.0,Low income,South Asia,201663.239336
13,Afghanistan,AFG,2003,0.0,8296.460291,1852.746488,4864.837685,5427.977558,113.253696,13399.17468,449.369180,33217.86158,9912.024674,2607.906847,2323.883437,387.327401,92.396359,179.896970,1160.444042,4570.640176,31901.53380,2211.928636,4398.482507,3649.958559,7095.064876,864.000077,1124.078720,13586.66102,511.795464,294.030425,137.000000,157.926775,2110.147100,54822.62001,1297.286529,3195.500654,163.0,Low income,South Asia,216377.216215


In [53]:
# Which year experienced the most deaths?

# In answering the above question, we group the data by year and total deaths in the year
# We then sum by years and sort our analysis by the total deaths column 
df_cleaned_data2[['year', 'total deaths']].groupby('year').sum().sort_values(by='total deaths', ascending=False).head(5)

,total deaths
year,
2017,1.606418e+07
2016,1.590923e+07
2015,1.554909e+07
2014,1.508075e+07
2013,1.467305e+07



# Question 3: What are the leading causes of death in the world for each year being analysed?



In [ ]:
df_cleaned_data2.groupby(['year']).sum()

,number of executions,road injuries,cirrhosis and other chronic liver diseases,digestive diseases,tuberculosis,hiv/aids,diarrheal diseases,intestinal infectious diseases,lower respiratory infections,meningitis,drowning,alzheimer disease and other dementias,parkinson disease,alcohol use disorders,drug use disorders,malaria,maternal disorders,neonatal disorders,nutritional deficiencies,diabetes mellitus,chronic kidney disease,chronic respiratory diseases,conflict and terrorism,hepatitis,neoplasms,"fire, heat, and hot substances",poisonings,exposure to forces of nature,environmental heat and cold exposure,protein-energy malnutrition,cardiovascular diseases,self-harm,interpersonal violence,terrorism deaths,total deaths
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2001,0.0,1.247010e+06,1.045063e+06,1.905335e+06,1.533826e+06,1.604589e+06,2.036660e+06,196021.971121,2.834489e+06,401476.647081,430191.153734,1.408364e+06,202081.522441,172530.925828,97738.909364,779612.590799,289988.745254,2.558311e+06,494581.040697,8.547376e+05,7.804339e+05,3.519484e+06,56966.214107,146746.292507,6.906796e+06,139173.047107,78825.607100,29055.251534,67842.392863,448247.285132,1.371005e+07,795849.420974,434264.341441,7579.0,4.721392e+07
2002,0.0,1.247378e+06,1.060341e+06,1.926777e+06,1.506788e+06,1.708773e+06,1.986958e+06,193003.763532,2.782093e+06,386682.695308,408684.421774,1.455375e+06,209892.401048,178278.478524,96462.610062,803225.828052,279872.340365,2.514485e+06,477849.038169,8.861766e+05,8.018280e+05,3.507198e+06,55878.024451,145244.957939,7.033976e+06,137558.285903,78969.056280,8274.989285,69912.930562,433260.998731,1.398804e+07,783310.441394,439279.119955,4610.0,4.759644e+07
2003,0.0,1.240676e+06,1.076333e+06,1.948802e+06,1.468948e+06,1.799349e+06,1.941984e+06,189929.514497,2.738296e+06,376998.098760,388542.080297,1.502855e+06,217703.660437,182859.494381,95173.756950,820347.019555,268258.472771,2.481009e+06,384539.339356,9.155863e+05,8.233985e+05,3.488901e+06,50540.821438,143297.048996,7.164347e+06,136933.509080,78918.523653,35316.205019,75716.514161,341317.784929,1.422912e+07,783974.782675,425506.278086,3089.0,4.781856e+07
2004,0.0,1.245221e+06,1.087676e+06,1.962781e+06,1.421983e+06,1.863073e+06,1.906548e+06,186855.237846,2.687492e+06,369147.608908,380394.020230,1.538959e+06,222292.450037,184656.135684,95926.105023,817300.075183,260807.562601,2.435878e+06,370578.214116,9.317492e+05,8.359237e+05,3.413772e+06,53844.156465,142962.584976,7.231325e+06,134169.375760,79294.666136,237554.077282,66947.758277,328565.598634,1.423384e+07,785516.986910,420707.148769,5689.0,4.793944e+07
2005,0.0,1.254934e+06,1.106150e+06,1.989690e+06,1.399388e+06,1.900634e+06,1.880861e+06,184168.825179,2.656818e+06,361407.466005,372753.584436,1.592255e+06,229451.589940,189614.260215,100443.081155,807088.444058,257552.072680,2.391310e+06,358158.662145,9.591532e+05,8.610229e+05,3.372921e+06,37257.865278,144766.523303,7.333573e+06,133723.490138,79691.863000,86914.636118,68898.966275,316658.408376,1.443513e+07,793523.196999,415006.746095,6279.0,4.807720e+07
2006,0.0,1.249398e+06,1.118230e+06,2.006585e+06,1.364166e+06,1.904661e+06,1.865905e+06,181140.132319,2.620732e+06,356019.061024,357242.661454,1.640160e+06,235172.571684,182684.733744,101291.717642,788156.433498,251071.164940,2.340450e+06,347206.142319,9.742642e+05,8.797903e+05,3.333411e+06,52737.849898,140455.489976,7.383927e+06,130291.799873,77422.734820,16741.733887,64322.779549,306200.248021,1.437835e+07,779701.084280,408003.336466,9337.0,4.784523e+07
2007,1248.0,1.254403e+06,1.129926e+06,2.025897e+06,1.334255e+06,1.871537e+06,1.842142e+06,178218.055583,2.593169e+06,347214.617243,348037.520516,1.698593e+06,242990.719453,178745.285899,102866.387840,772434.024816,244572.159048,2.287356e+06,336792.280879,9.929854e+05,9.037167e+05,3.339236e+06,55751.148755,138047.036675,7.507107e+06,128477.413013,76165.770272,15405.512778,60901.741026,296094.176389,1.451232e+07,773121.918016,399144.751170,12637.0,4.800150e+07
2008,671.0,1.265474e+0

From the analysis above, the leading causes of death across the years in the dataset are from: road injuries, cirrhosis and other chronic liver diseases, digestive diseases, tuberculosis, hiv/aids, diarrheal diseases, lower respiratory infections, alzheimer disease and other dementias, neonatal disorders, diabetes mellitus, chronic kidney disease, chronic respiratory diseases, neoplasms and cardiovascular diseases.

# Question 4: Which region and income group registered the most deaths?


In [ ]:
# Which region experienced the most deaths?

# In answering the above question, we group the data by region and total deaths in the year
# We then sum by region and sort our analysis by the total deaths column 
df_cleaned_data2[['region', 'total deaths']].groupby('region').sum().sort_values(by='total deaths', ascending=False).head(20)

,total deaths
region,
East Asia & Pacific,2.442567e+08
South Asia,1.886232e+08
Europe & Central Asia,1.531082e+08
Sub-Saharan Africa,1.238426e+08
Latin America & Caribbean,5.395461e+07
North America,4.574328e+07
Middle East & North Africa,2.948002e+07


In [ ]:
# Which income group experienced the most deaths?

# In answering the above question, we group the data by income group and total deaths in the year
# We then sum by income group and sort our analysis by the total deaths column 
df_cleaned_data2[['income group', 'total deaths']].groupby('income group').sum().sort_values(by='total deaths', ascending=False).head(20)

,total deaths
income group,
Lower middle income,3.238157e+08
Upper middle income,2.438759e+08
High income: OECD,1.498771e+08
Low income,7.147850e+07
High income: nonOECD,4.785541e+07
Others,2.106007e+06


# Question 5: What were the leading causes of deaths per region and income group?




In [ ]:
# What are the leading causes of death per region?

df_cleaned_data2[addition_columns].groupby(df_cleaned_data2['region']).sum().head(3)


,number of executions,road injuries,cirrhosis and other chronic liver diseases,digestive diseases,tuberculosis,hiv/aids,diarrheal diseases,intestinal infectious diseases,lower respiratory infections,meningitis,drowning,alzheimer disease and other dementias,parkinson disease,alcohol use disorders,drug use disorders,malaria,maternal disorders,neonatal disorders,nutritional deficiencies,diabetes mellitus,chronic kidney disease,chronic respiratory diseases,conflict and terrorism,hepatitis,neoplasms,"fire, heat, and hot substances",poisonings,exposure to forces of nature,environmental heat and cold exposure,protein-energy malnutrition,cardiovascular diseases,self-harm,interpersonal violence,terrorism deaths
region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
East Asia & Pacific,765.0,7.341976e+06,5.757868e+06,9.762497e+06,4.357860e+06,1.439988e+06,2.156449e+06,423039.149491,9.085918e+06,536956.626511,2.071347e+06,1.094101e+07,1.527446e+06,4.661599e+05,404956.074114,279825.883502,400629.701241,4.225126e+06,716466.245019,5.095915e+06,4.974278e+06,2.366059e+07,36001.801094,385938.366346,5.372863e+07,343365.987057,529479.882342,482996.580143,137559.782036,616882.517687,8.761507e+07,3.875104e+06,8.700142e+05,8598.0
Europe & Central Asia,19.0,1.951714e+06,3.660976e+06,6.973241e+06,7.540267e+05,4.624765e+05,2.529411e+05,7449.097973,4.207754e+06,125997.022407,5.622564e+05,9.653360e+06,1.238654e+06,1.417223e+06,449927.664437,553.817373,35549.721769,9.669154e+05,112114.665105,2.399148e+06,2.086781e+06,6.142932e+06,44686.326533,58059.311936,3.397369e+07,427368.981016,299549.024843,7296.974282,532208.610461,75171.227521,7.048060e+07,2.800449e+06,9.381908e+05,8966.0
Latin America & Caribbean,0.0,1.964523e+06,1.787159e+06,3.225280e+06,3.442317e+05,8.126687e+05,5.634903e+05,3694.106844,3.153892e+06,154674.892495,3.453854e+05,2.563811e+06,3.485316e+05,3.498164e+05,86317.554619,4822.134352,152845.600037,1.660297e+06,438921.637742,2.464236e+06,2.218551e+06,2.723300e+06,15623.000062,52104.411327,9.567645e+06,114313.330690,48161.183345,248629.625309,25061.766777,397899.157407,1.491407e+07,6.979678e+05,2.503692e+06,2991.0


In [ ]:
# What are the leading causes of death per income group?

df_cleaned_data2[addition_columns].groupby(df_cleaned_data2['income group']).sum()

,number of executions,road injuries,cirrhosis and other chronic liver diseases,digestive diseases,tuberculosis,hiv/aids,diarrheal diseases,intestinal infectious diseases,lower respiratory infections,meningitis,drowning,alzheimer disease and other dementias,parkinson disease,alcohol use disorders,drug use disorders,malaria,maternal disorders,neonatal disorders,nutritional deficiencies,diabetes mellitus,chronic kidney disease,chronic respiratory diseases,conflict and terrorism,hepatitis,neoplasms,"fire, heat, and hot substances",poisonings,exposure to forces of nature,environmental heat and cold exposure,protein-energy malnutrition,cardiovascular diseases,self-harm,interpersonal violence,terrorism deaths
income group,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
High income: OECD,442.0,1.919082e+06,3.161909e+06,6.686083e+06,2.445837e+05,2.937634e+05,3.831765e+05,1.014250e+03,5.803971e+06,8.018641e+04,3.267884e+05,1.390118e+07,1.627562e+06,650250.418899,826865.378665,2.938640e+02,2.941438e+04,4.780989e+05,1.803780e+05,3.211462e+06,3.350876e+06,8.317121e+06,20630.717574,8.548672e+04,4.276713e+07,261197.209378,75466.560707,33179.255504,128240.505477,1.438696e+05,5.162470e+07,2.773231e+06,4.841981e+05,5295.0
High income: nonOECD,1032.0,1.128764e+06,1.123242e+06,2.074791e+06,3.943714e+05,3.281884e+05,6.091287e+04,2.804774e+03,1.416292e+06,5.558447e+04,2.931915e+05,1.789263e+06,2.434409e+05,736728.597509,226698.540984,1.936716e+04,2.601956e+04,3.847958e+05,5.022691e+04,5.955447e+05,5.863490e+05,1.313161e+06,21254.721991,1.449019e+04,7.847342e+06,225148.811716,191516.156680,2952.861774,348626.662207,3.679584e+04,2.418801e+07,1.234513e+06,8.902293e+05,3764.0
Low income,329.0,1.610070e+06,1.283813e+06,2.392676e+06,3.920005e+06,8.980209e+06,5.528204e+06,2.358180e+05,6.743647e+06,1.636062e+06,4.697413e+05,7.347086e+05,1.137547e+05,79069.995758,41371.784216,4.903822e+06,9.492804e+05,7.124095e+06,1.956744e+06,1.097278e+06,9.280379e+05,2.294351e+06,355501.991037,1.972584e+05,4.710738e+06,246246.327976,112806.022584,254187.307779,52901.315099,1.896178e+06,9.468250e+06,5.589973e+05,5.407876e+05,61565.0
Lower middle income,1189.0,8.073118e+06,9.158991e+06,1.534198e+07,1.521428e+07,1.047549e+07,2.273947e+07,2.445361e+06,2.214771e+07,3.362869e+06,2.859273e+06,5.669302e+06,1.029446e+06,783923.924582,355010.056916,6.832167e+06,2.622506e+06,2.407010e+07,2.768264e+06,7.804512e+06,6.488536e+06,2.592656e+07,691835.675046,1.713051e+06,2.900710e+07,949117.091759,343273.150401,537442.844957,391183.301067,2.272484e+06,8.455534e+07,5.023081e+06,2.065021e+06,96727.0
Others,0.0,1.839641e+04,2.489835e+04,7.269107e+04,4.012293e+03,1.185692e+03,7.331398e+02,9.822401e+00,2.222120e+04,7.349727e+02,2.755318e+03,8.694402e+04,1.320991e+04,7355.812959,1861.627315,0.000000e+00,2.501539e+02,8.317520e+03,3.105217e+02,5.888059e+04,3.801513e+04,7.133134e+04,130.000000,2.050573e+02,4.597980e+05,2314.353217,950.681445,89.000000,1156.054294,1.674211e+02,1.168122e+06,3.357183e+04,5.383226e+03,4.0
Upper middle income,5064.0,8.442063e+06,4.986855e+06,8.932298e+06,2.331926e+06,6.061095e+06,1.429407e+06,1.098024e+05,7.988523e+06,5.067827e+05,1.889987e+06,9.840571e+06,1.476156e+06,755352.614991,537830.702951,1.986938e+05,3.151949e+05,4.969708e+06,7.318312e+05,5.471766e+06,5.293615e+06,2.181654e+07,341143.899034,2.969165e+05,5.071727e+07,475537.268457,563580.857354,175889.164617,172381.970416,6.442381e+05,8.972268e+07,3.632865e+06,2.947548e+06,94780.0


# Question 6: Which country registered the most deaths?

In [54]:
df_cleaned_data2[['country', 'total deaths']].groupby('country').sum().sort_values(by='total deaths', ascending=False).head(10)

,total deaths
country,
China,1.494157e+08
Brazil,1.874724e+07
Bangladesh,1.424243e+07
Argentina,5.055897e+06
Canada,3.962217e+06
Afghanistan,3.553086e+06
Colombia,3.379635e+06
Cote d'Ivoire,3.274969e+06
Cameroon,3.198079e+06
